<img src="logo.png">


___

<h1>
<p style="text-align: center;">Tablas de DynamoDB <p>
</h1>

### Primero, necesitamos instalar algunas dependencias:

In [ ]:
#libreria aws, simplificar el desarrollo de python , framework
!pip install boto3 cython PyHamcrest 


### Ahora, tenemos que realizar algunas importaciones::

In [ ]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import sys
import random
from time import sleep
from boto3.dynamodb.conditions import Key, Attr
import pandas as pd

### Creemos nuestra tabla de 'películas' y el índice secundario local:

In [ ]:
dynamodb = boto3.resource('dynamodb',  region_name='us-east-2')


table = dynamodb.create_table(
        TableName= 'movies',
        KeySchema=[
            {
                'KeyType': 'HASH',
                'AttributeName': 'year'
            },
            {
                'KeyType': 'RANGE',
                'AttributeName': 'title'
            }
        ],
        LocalSecondaryIndexes=[
            {
                'IndexName': 'year-actor-index',
                'KeySchema': [
                    {
                        'KeyType': 'HASH',
                        'AttributeName': 'year'
                    },
                    {
                        'KeyType': 'RANGE',
                        'AttributeName': 'actor'
                    }
                ],

                'Projection': {
                    'ProjectionType': 'ALL',
                }
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'actor',
                'AttributeType': 'S'
            }            
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 2,
            'WriteCapacityUnits': 2
        }
    )
# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='movies')
print('Table is ready, please continue as instructed.')

### Subamos algunos datos:

In [ ]:
dynamodb = boto3.resource('dynamodb',  region_name='us-east-2')
table = dynamodb.Table('movies')
table.put_item(
    Item={
        'year': 2005,
        'title': 'Batman Begins',
        'actor': 'Christian Bale'
    }
)
table.put_item(
    Item={
        'year': 2008,
        'title': 'The Dark Knight Rises',
        'actor': 'Christian Bale'
    }
)
table.put_item(
    Item={
        'year': 2008,
        'title': 'Tropic Thunder',
        'actor': 'Robert Downey Jr.'
    }
)
table.put_item(
    Item={
        'year': 2008,
        'title': 'Iron Man',
        'actor': 'Robert Downey Jr.'
    }
)

response = table.scan()

for i in response['Items']:
    print("added item:", i['year'], ":", i['title'], ":", i['actor'])

### Ahora, realicemos algunas consultas:

##### Primero, realizaremos un escaneo de nuestra tabla:

In [ ]:
dynamodb = boto3.resource('dynamodb',  region_name='us-east-1', )
table = dynamodb.Table('movies')

response = table.scan()

item_list = []
for i in response['Items']:
    item = {'year':i['year'], 'title':i['title'], 'actor' :i['actor']}
    item_list.append(item)
df = pd.DataFrame(data=item_list)
df

##### Ahora, realicemos una consulta para todas las películas lanzadas en 2008 con títulos que comienzan con "T":

In [ ]:
dynamodb = boto3.resource('dynamodb',  region_name='us-east-1')
table = dynamodb.Table('movies')

response = table.query(
    KeyConditionExpression=Key('year').eq(2008) & Key('title').begins_with('T')
)

item_list = []
for i in response['Items']:
    item = {'year':i['year'], 'title':i['title'], 'actor' :i['actor']}
    item_list.append(item)
df = pd.DataFrame(data=item_list)
df

#### A continuación, consultaremos el índice secundario local para todas las películas lanzadas por Robert Downey Jr. en 2008:

In [ ]:
dynamodb = boto3.resource('dynamodb',  region_name='us-east-1', )
table = dynamodb.Table('movies')


response = table.query(
    IndexName = 'year-actor-index',
    KeyConditionExpression=Key('year').eq(2008) & Key('actor').eq('Robert Downey Jr.')
    
)

item_list = []
for i in response['Items']:
    item = {'year':i['year'], 'title':i['title'], 'actor' :i['actor']}
    item_list.append(item)
df = pd.DataFrame(data=item_list)
df


# ¡Todo terminado! ¡Buen trabajo!
<img src="buen_Trabajo.jpg">
